# Feature Selection

We will go through some of the basic techinques for dealing with feature selection. 
We will cover:

- Filter Methods/Uni-variate Selection Methods
- Wrapper Methods

In [3]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load Data

We'll be working with a Bank Churn dataset. Churn Prediction is a common problem for **all** companies (not just tech companies). Having a healthy business is a result of minimizing churn as much as possible. 


In [4]:
df = pd.read_csv("../data/BankChurners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   CLIENTNUM                                                                                                                           10127 non-null  int64  
 1   Attrition_Flag                                                                                                                      10127 non-null  object 
 2   Customer_Age                                                                                                                        10127 non-null  int64  
 3   Gender                                                                           

In [6]:
df.isnull().sum()

CLIENTNUM                                                                                                                             0
Attrition_Flag                                                                                                                        0
Customer_Age                                                                                                                          0
Gender                                                                                                                                0
Dependent_count                                                                                                                       0
Education_Level                                                                                                                       0
Marital_Status                                                                                                                        0
Income_Category                                 

## Pre-processing

The first thing we need to do pre-process our dataset. In this dataset, we need to turn the `Attrition_Flag` into a dummy variable of 0 and 1. Otherwise, we won't be able to make a proper prediction. 

In [7]:
# df1 = df.copy()

In [8]:
# dum = pd.get_dummies(df1['Attrition_Flag'])

In [9]:
# df1 = pd.concat([df1, dum], axis=1).drop(columns="Attrition_Flag", axis=1)

In [10]:
# df1.head()

In [11]:
def make_target(df, column):
    target_dum = pd.get_dummies(df[column])
    df = pd.concat([df, target_dum], axis= 1)
    return df

In [12]:
## To keep things simple, we'll just use the int columns as the feature columns. 
def get_int_cols(df, dtype):
    features = []
    for col, t in zip(df.columns, list(df.dtypes)):
        if t == dtype:
            features.append(col)

    return features




In [13]:
df = make_target(df, column='Attrition_Flag')

In [14]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,Attrited Customer,Existing Customer
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991,False,True
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994,False,True
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998,False,True
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987,False,True
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998,False,True


In [15]:
target = "Attrited Customer"

features = get_int_cols(df, 'int64')

In [16]:
X = df[features]
y = df[target]

In [17]:
X.shape

(10127, 10)

In [18]:
y.shape

(10127,)

In [19]:
y.head()

0    False
1    False
2    False
3    False
4    False
Name: Attrited Customer, dtype: bool

## Filter Methods (Univariate Feature Selection)

### Correlation/ANOVA

The first method we'll go over is correlation/ANOVA. This is simply where we just select features that have the highest correlation with the target variable. For this dataset, since we're predicting churn, we'll be using ANOVA, as ANOVA is how we measure correlation for categorical variables. 

Here the arguments can be defined as: 

`df`: dataframe input

`features`: list of features within the dataframe you want to evaluate

`target`: the target variable you want to run the correlation against

`threshold`: The correlation threshold. IF you set it to 0.10, it'll return all features that have a correlation greater than 0.10. 

In [20]:
correlation_threshold = 0.10

def correlation_selection(df, features, target, threshold):
    corr = df[features + [target]].corr()[target]

    select_features = corr[abs(corr) > correlation_threshold]

    remove_target = select_features.index[select_features.index != target]
    return select_features[remove_target]


select = correlation_selection(df, features, target, 0.10)


In [21]:
print(select)

Total_Relationship_Count   -0.150005
Months_Inactive_12_mon      0.152449
Contacts_Count_12_mon       0.204491
Total_Revolving_Bal        -0.263053
Total_Trans_Amt            -0.168598
Total_Trans_Ct             -0.371403
Name: Attrited Customer, dtype: float64


### Chi-Squares, ANOVA, F-Test, Mutual Info Gain

In addition to correlation, there are a number of other uni-variate methods. Sklearn has an implementation called `SelectKBest` which selects features according to the k highest scores. Sklearn has a ton of different options here:

`chi2`: Chi-Squared statistics comparing features against categorical target. 

`f_regression`: F-statistic between the feature and the target.

`f_classif`: ANOVA F-value between feature and target. 

`r_regression`: Pearson Correlation. Similar to the previous cell. 

`mutual_info_classif`: Mutual information for a discrete target.

`mutual_info_regression`: Mutual information for a continuous target.

In [23]:
# import all highlighted libraries above

from sklearn.feature_selection import chi2, SelectKBest, f_classif, f_regression, mutual_info_classif, mutual_info_regression

kb = SelectKBest(chi2, k=4)

X_new = kb.fit_transform(X, y)

X_new

array([[768805383,       777,      1144,        42],
       [818770008,       864,      1291,        33],
       [713982108,         0,      1887,        20],
       ...,
       [716506083,         0,     10291,        60],
       [717406983,         0,      8395,        62],
       [714337233,      1961,     10294,        61]])

In [24]:
# Create a df from xnew
X_new = pd.DataFrame(X_new)
X_new

,0,1,2,3
0,768805383,777,1144,42
1,818770008,864,1291,33
2,713982108,0,1887,20
3,769911858,2517,1171,20
4,709106358,0,816,28
...,...,...,...,...
10122,772366833,1851,15476,117
10123,710638233,2186,8764,69
10124,716506083,0,10291,60
10125,717406983,0,8395,62


In [25]:
# give column name to xnew
X_new.columns = kb.get_feature_names_out()
X_new

,CLIENTNUM,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct
0,768805383,777,1144,42
1,818770008,864,1291,33
2,713982108,0,1887,20
3,769911858,2517,1171,20
4,709106358,0,816,28
...,...,...,...,...
10122,772366833,1851,15476,117
10123,710638233,2186,8764,69
10124,716506083,0,10291,60
10125,717406983,0,8395,62


# Wrapper method

### Forward Stepwise

Next, we'll use forward stepwise. We can use sklearn's implementation as well, called `SequentialFeatureSelector`. We'll need to just speciy this in the parameters: 

`estimator`: This is the estimator/model we want to evaluate the features on. You can input random forest or logistic regression or any model you'd use for your problem. 

`n_features_to_select`: The number of features we want to select. 

`direction`: Denote `'forward'` for forward stepwise. Denote `'backward'` for backward stepwise. 



In [26]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression


sfs = SequentialFeatureSelector(LogisticRegression(),
                                n_features_to_select=4,
                                direction="forward")

x_new = sfs.fit_transform(X, y)
x_new

/home/jagannath/MLProcess/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[45,  3, 39,  3],
       [49,  5, 44,  2],
       [51,  3, 36,  0],
       ...,
       [44,  1, 36,  4],
       [30,  2, 36,  3],
       [43,  2, 25,  4]])

In [27]:
x_new = pd.DataFrame(x_new)
x_new

,0,1,2,3
0,45,3,39,3
1,49,5,44,2
2,51,3,36,0
3,40,4,34,1
4,40,3,21,0
...,...,...,...,...
10122,50,2,40,3
10123,41,2,25,3
10124,44,1,36,4
10125,30,2,36,3


In [28]:
x_new.columns = sfs.get_feature_names_out()
x_new

,Customer_Age,Dependent_count,Months_on_book,Contacts_Count_12_mon
0,45,3,39,3
1,49,5,44,2
2,51,3,36,0
3,40,4,34,1
4,40,3,21,0
...,...,...,...,...
10122,50,2,40,3
10123,41,2,25,3
10124,44,1,36,4
10125,30,2,36,3


### Backward Stepwise

Forward backward stepwise, we'll just change the argument `direction` to equal `'backward'`:;

## Recursive Feature Elimination

Recursive Feature Elimination is where we train the model on all features. Then, we prune the least important features. Rerun the model. Prune features again. We keep repeating this until we hit our desired feature amount.

Recursive Feature Elimination and Sequential are very similar. The difference is Sequential is using the cross validation score to remove features. RFE is using the importance or weight of the feature to decide on removal:

In [29]:
from sklearn.feature_selection import RFE

# now like we did above

rfe = RFE(LogisticRegression(), n_features_to_select=4)
rfe.fit_transform(X, y)
rfe

/home/jagannath/MLProcess/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jagannath/MLProcess/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

RFE(estimator=LogisticRegression(), n_features_to_select=4)

In [30]:
xnew = rfe.fit_transform(X, y)
xnew

/home/jagannath/MLProcess/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jagannath/MLProcess/ml_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

array([[3, 5, 1, 3],
       [5, 6, 1, 2],
       [3, 4, 1, 0],
       ...,
       [1, 5, 3, 4],
       [2, 4, 3, 3],
       [2, 6, 2, 4]])

In [31]:
xnew = pd.DataFrame(xnew)
xnew.columns = rfe.get_feature_names_out()

xnew

,Dependent_count,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon
0,3,5,1,3
1,5,6,1,2
2,3,4,1,0
3,4,3,4,1
4,3,5,1,0
...,...,...,...,...
10122,2,3,2,3
10123,2,4,2,3
10124,1,5,3,4
10125,2,4,3,3
